# Phillips 2 layers model example

In [ ]:
# import modules
from IPython import display
import warnings
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

## 0. Load the data from the previously run model:

In [ ]:
phillips = "~/MOM6-examples/ocean_only/Phillips_2layer"

# snapshots
ds = xr.open_dataset(f"{phillips}/prog.nc")

# initial conditions
ic = xr.open_dataset(f"{phillips}/MOM_IC.nc")
ic = ic.rename({"lonh": "xh", "lath": "yh",
                "lonq": "xq", "latq": "yq",
                "Layer": "zl",
                "Interface": "zi"})

In [ ]:
# add grid metrics
dsgrid = xr.open_dataset(f"{phillips}/ocean_geometry.nc")
dsgrid = dsgrid.rename({"lonh": "xh", "lath": "yh",
                        "lonq": "xq", "latq": "yq"})
ds = xr.merge([ds, dsgrid])

In [ ]:
ds

## 1. Plot initial conditions

In [ ]:
ic

Compute the zonal mean:

In [ ]:
ic_zonalmean = ic.mean(dim=["xh", "xq"]).squeeze(dim="Time")

In [ ]:
# make 2d depth/lat latitude array 
ic_zonalmean["lat"] = ic_zonalmean["yh"].broadcast_like(ic_zonalmean["eta"])

In [ ]:
plt.figure()
ax=plt.axes()
ic_zonalmean["eta"].plot.line(ax=ax, x="yh", color="k")

# using yh as coord, first u-point is out of bounds
C = ax.pcolormesh(ic_zonalmean["lat"].values,
                  ic_zonalmean["eta"].values,
                  ic_zonalmean["u"].values[:,1:],
                  cmap="bwr", vmin=-0.5, vmax=0.5)
plt.colorbar(C)
plt.plot()

## 2. Grid-dependent diagnostics with xgcm


https://xgcm.readthedocs.io/en/latest/

Quick reminder: MOM6 is a C-grid model

In [ ]:
display.Image("https://mom6.readthedocs.io/en/main/_images/Arakawa_C_grid.png")

The grid metrics are defined as:

In [ ]:
display.Image("https://mom6.readthedocs.io/en/main/_images/Grid_metrics.png")

### 2.1 Defining a xgcm grid object

In [ ]:
from xgcm import Grid

xgrid = Grid(ds, coords={'X': {'center': 'xh', 'outer': 'xq'},
                         'Y': {'center': 'yh', 'outer': 'yq'},
                         'Z': {'center': 'zl', 'outer': 'zi'} },
             periodic=['X'])

### 2.2 Velocity at tracer-point

In [ ]:
speed2 = (xgrid.interp(ds.u, "X", boundary="extend")**2 +
          xgrid.interp(ds.v, "Y", boundary="extend")**2)

In [ ]:
speed2

In [ ]:
speed2.sel(Time="0001-04-11", zl=1031).plot(cmap="Reds", vmin=0., vmax=2.)

### Exercice: how would you compute it on the vorticity (q) point?

In [ ]:
speed2_q = (xgrid.interp(ds.u, "Y", boundary="extend")**2 +
            xgrid.interp(ds.v, "X", boundary="extend")**2)

In [ ]:
speed2_q

In [ ]:
speed2_q.sel(Time="0001-04-11", zl=1031).plot(cmap="Reds")

### 2.3 Relative vorticity

$$\zeta = \dfrac{\partial v}{\partial x} - \dfrac{\partial u}{\partial y} $$

In [ ]:
rel_vort = ((xgrid.diff(ds.v, 'X', boundary='extend') / ds.dxBu) -
            (xgrid.diff(ds.u, 'Y', boundary='extend') / ds.dyBu))

In [ ]:
fig, axs = plt.subplots(ncols=2,figsize=[8,4])
rel_vort.isel(Time=-1, zl=0).plot(ax=axs[0], add_labels=False)
ds["RV"].isel(Time=-1, zl=0).plot(ax=axs[1], add_labels=False)
axs[0].set_title("$\zeta$ xgcm")
axs[1].set_title("$\zeta$ model")

### 2.4 Exercice: compute the potential vorticity (layer) and compare with model output

$$PV = \dfrac{(f + \zeta)}{h}$$

In [ ]:
h_u = xgrid.interp(ds.h, 'X', boundary='extend')
h_q = xgrid.interp(h_u, "Y", boundary='extend')

pot_vort = (ds.f + rel_vort) / h_q

In [ ]:
fig, axs = plt.subplots(ncols=2,figsize=[8,4])
pot_vort.isel(Time=-1, zl=0).plot(ax=axs[0], add_labels=False,
                                  vmin=0, vmax=6.e-7, cmap="coolwarm")
ds["PV"].isel(Time=-1, zl=0).plot(ax=axs[1], add_labels=False,
                                  vmin=0, vmax=6.e-7, cmap="coolwarm")
axs[0].set_title("$PV$ xgcm")
axs[1].set_title("$PV$ model")

### 2.5 Timeseries of enstrophy

In [ ]:
tot_enstrophy = (rel_vort*rel_vort*ds.Aq).sum(dim=["xq", "yq"])

In [ ]:
ax = plt.axes()
tot_enstrophy.plot.line(ax=ax, x="Time")
ax.grid()
ax.set_title("Total enstrophy $\zeta^{2}$ ($m^{2}.s^{-1}$)")
plt.show()

### Exercice: repeat for potential vorticity anomaly to initial condition

In [ ]:
tot_PV = (pot_vort*ds.Aq).sum(dim=["xq", "yq"])

In [ ]:
ax = plt.axes()
(tot_PV -tot_PV.isel(Time=0)).plot.line(ax=ax, x="Time")
ax.grid()
ax.set_title("Total PV anomaly to initial conditions ($m.s^{-1}$)")
plt.show()

Homework: read **Hallberg, Ocean Modeling, 2013** (https://www.sciencedirect.com/science/article/pii/S1463500313001601?casa_token=HOdzr7PUELUAAAAA:UKvTM7_FouIiITlAaexi4PrQSVTcvJxdTCCiMd1-WNevquJqVWHtpYlCMxc3mP8Whycwla8ztg) and reproduce the figures from the article